In [1]:
import nibabel as nib
from nilearn import plotting
import matplotlib.pyplot as plt
from utils.visualization_tools import *
import utils.visualization_tools as visualization_tools
from utils.data_processor import *

/usr/local/lib/python3.6/dist-packages/nilearn/datasets/__init__.py:96: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  "Numpy arrays.", FutureWarning)


In [2]:
import pandas as pd
import os
from tqdm import tqdm

In [3]:
brain_to_exclude = 'n20'
allowed_subdirs = [f"sub-{name[:-4]}" for name in os.listdir(f"raw_data/normalized_label")]
brains = [[f"raw_data/output/fmriprep/{subdir}/anat/{name}" for name in os.listdir(f"raw_data/output/fmriprep/{subdir}/anat") if name.endswith("Asym_desc-preproc_T1w.nii.gz")] for subdir in os.listdir(f"raw_data/output/fmriprep/") if subdir.count(".") == 0 and subdir.startswith("sub") and subdir in allowed_subdirs]
brains = [item for sublist in brains for item in sublist]
brain_nums = [x.split('/')[3][4:] for x in brains]
i = brain_nums.index(brain_to_exclude)
del brain_nums[i]


brains = [f'raw_data/nii_brains/{num}.nii' for num in brain_nums]
labels = [f'raw_data/nii_labels/{num}.nii' for num in brain_nums]
brains_loaded = [nib.load(brain) for brain in tqdm(brains)]
labels_loaded = [nib.load(label) for label in tqdm(labels)]

100%|██████████| 91/91 [00:01<00:00, 66.14it/s]


In [4]:
def average_prediction(pred,brain):
    pred_brain = pred * (brain > 0)
    pred_sum = pred_brain.sum()
    brain_sum = (brain > 0).sum()
    return pred_sum / brain_sum
def fcd_proportion(mask,brain):
    fcd_brain = (mask >0) * (brain > 0)
    brain_sum = (brain > 0).sum()
    fcd_sum = (fcd_brain > 0).sum()
    return fcd_sum / brain_sum
def is_brain_in_crop(brain):
    if brain.max()>0:
        return 1
    else:
        return 0
def fcd_total_number_of_points(mask):
    return mask.sum()


top1 = []
top3 = []
top5 = []
for brain in range(91):
    print(brain)
    pred_path = f'interpolate/results/brain_{brain}/griddata/2021-10-16_16bs_abscoords_air_cropsize64_epochs400_whole.nii.gz'

    pred = load_nii_to_array(pred_path)
    mask = labels_loaded[brain].get_fdata()
    brain = brains_loaded[brain].get_fdata()


    av_prs = []
    fcd_props = []
    is_brains = []
    fcd_totals = []
    step = 64
    win_size = 64
    x = 0
    y = 0
    z = 0
    while True:
        while True:
            while True:
#                 try:
                brain_crop = brain[x:x+win_size,y:y+win_size,z:z+win_size]
                mask_crop = mask[x:x+win_size,y:y+win_size,z:z+win_size]
                pred_crop = pred[x:x+win_size,y:y+win_size,z:z+win_size]
                av_pr = average_prediction(pred_crop,brain_crop)
                fcd_prop = fcd_proportion(mask_crop,brain_crop)
                is_brain = is_brain_in_crop(brain_crop)
                fcd_total = fcd_total_number_of_points(mask_crop)

                av_prs.append(av_pr)
                fcd_props.append(fcd_prop)
                is_brains.append(is_brain)
                fcd_totals.append(fcd_total)

#                 except Exception:
#                     pass
                if z + win_size < brain.shape[2]:
                        z+=win_size
                else:
                    z = 0
                    break
            if y + win_size < brain.shape[1]:
                y+=win_size
            else:
                y = 0
                break
        if x + win_size < brain.shape[0]:
            x+=win_size
        else:
            x = 0
            break
    df = pd.DataFrame({'is_brain':is_brains,
                       'proba':av_prs,
                       'fcd_prop':fcd_props,
                       'num_points_fcd':fcd_totals})
    df = df.dropna().sort_values('proba',ascending = False)
    df['place'] = range(1, df.shape[0]+1)
#     display(df[df.fcd_prop>0])
    if df[df.fcd_prop>0].place.min() <= 5:
        top5.append(1)
        if df[df.fcd_prop>0].place.min() <= 3:
            top3.append(1)
            if df[df.fcd_prop>0].place.min() <= 1:
                top1.append(1)
            else:
                top1.append(0)
        else:
            top3.append(0)
            top1.append(0)
    else:
        top5.append(0)
        top3.append(0)
        top1.append(0)
    del pred,mask,brain

0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in long_scalars
  # Remove the CWD from sys.path while we load stuff.


1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in long_scalars
  # Remove the CWD from sys.path while we load stuff.


2


...

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in long_scalars
  # Remove the CWD from sys.path while we load stuff.


89


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in long_scalars
  # Remove the CWD from sys.path while we load stuff.


90


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in long_scalars
  # Remove the CWD from sys.path while we load stuff.


In [8]:
for num in top5:
    print(str(num))

0
0
0
0
0
0
0
0
0
0
0
1
1
0
1
1
1
1
0
0
0
0
0
0
0
0
0
0
1
1
1
0
0
1
0
1
1
0
0
0
0
0
0
1
0
0
0
0
0
1
0
0
1
0
0
1
0
1
0
0
0
1
1
0
1
0
0
0
0
0
0
0
1
0
0
1
0
0
0
0
0
0
1
0
1
1
0
0
1
1
1


In [5]:
df[df.fcd_prop>0]

,is_brain,proba,fcd_prop,num_points_fcd,place
102,1,0.622981,0.002065,392.249572,4
12,1,0.481994,0.003155,1180.044194,10
42,1,0.473503,0.000237,324.215321,11
72,1,0.471158,0.001995,2042.036324,12
48,1,0.403006,0.001328,374.270675,15
43,1,0.399490,0.003745,1994.552271,16
77,1,0.391271,0.004534,3865.790541,17
67,1,0.383231,0.000602,462.256091,18
51,1,0.380303,0.000680,277.888634,20
47,1,0.361026,0.002048,2052.732029,23
